In [1]:
#William Dahl
#CSI 436
#Final Project
#May 9th, 2019

#This is a a Emoji recomendation app that uses Linear SVM to recomend emojis to a user based on a given message

from collections import Counter
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import pickle
from pathlib import Path
from tkinter import *
import numpy as np

#creates our model and our Vectorizer to be used
model =  LinearSVC(max_iter=10000)
vector = DictVectorizer()

#breaks mesages down into tokens of a specific length
def tokens(mesg, length):
    words = mesg.split()
    phrases = []
    for i in range(length-1, len(words)): 
        phrase = ' '.join(words[i-length+1:i+1])
        phrases.append(phrase) 
        
    return phrases

#gives lables to each of the classes of the messages int he data
def classify(label):
    #our lables
    feelings = ["joy", 'fear', "anger", "sadness", "disgust", "shame", "guilt"]
    clas = ""
    #maps the vector from strings to floats
    lables = list(map(float, label.split()))
    #maps the vectors to their respective class
    for i in range(len(lables)): 
        if lables[i] == 1: 
            clas += feelings[i] + " "

    return clas

#extracts the features from each message in the form of n-grams
def getFeatures(mesg):
    mesg = mesg.lower()
    features = [] #holds a list of the extracted features
    #makes 2-gram to 4-grams form the message
    for i in range(1, 5): 
        features += tokens(mesg, i)

    #Makes 1-grams from the message
    features += tokens(mesg, 1)
    #returns the features as dictinary with the n-gram as the keys
    #and the number of times it appears in other n-grams extracted from the message as the value
    return Counter(features)

#recomends a emoji to the user based on thier input
def Recomend_Emoji():
    global model #imports our model
    global vector #imports are vectorizer
    #mapping from the class to a emoji
    emoji_map = {"joy ":"😂", "fear ":"😱", "anger ":"😠", "sadness ":"😢", "disgust ":"😒", "shame ":"😳", "guilt ":"😳"}
    mesg_dict = getFeatures(mesg.get()) #gets features of the message
    mesg_dict = vector.transform(mesg_dict) #vectorizes the features
    emoji = model.predict(mesg_dict)[0]#predicts the entiment
    print("Recomended emoji: ", emoji_map[emoji])#writes the recomended emoji out

#Checks if the file model.sav exsists
config = Path('model.sav')
#if so it imports the model and the vetorizer form the respective files
if config.is_file():
    model = pickle.load(open("model.sav", 'rb'))
    vector = pickle.load(open("vector.sav", 'rb'))

#other wise it begeins traingin the model and the vectorizer
else:
    mesgs = [] #messges
    classes = [] #classes of the messages
    data  = [] #data matrix
    
    #reads the data file
    for line in open("data.txt", 'r'): 
        mesg = line[line.find("]")+1:]
        clas = ' '.join(line[1:line.find("]")].split())
        data.append([clas, mesg])
    
    #creates a list of dictanaries with n-grams in it for each message
    #creates a vector of labels for each message
    for clas, mesg in data:
        mesgs.append(getFeatures(mesg))
        classes.append(classify(clas))

    #splits the training and testing set
    mesgs_train, mesgs_test, classes_train, classes_test = train_test_split(mesgs, classes, test_size = 0.2)
    #fits the vectorizer to the training data
    mesgs_train = vector.fit_transform(mesgs_train)
    #vectorizes the testing data
    mesgs_test = vector.transform(mesgs_test)
    #fits the model with the training data
    model.fit(mesgs_train, classes_train)
    #gets the accuracy of the model
    score = accuracy_score(classes_test, model.predict(mesgs_test))
    print("testing accuracy:", score)

    #re fits the model with all of the data 
    model.fit(vector.fit_transform(mesgs), classes)
    #dumps the files modle.sava and vector.sav with the model and the vectorizer respectivly
    pickle.dump(model, open("model.sav", 'wb'))
    pickle.dump(vector, open("vector.sav", 'wb'))

#Creates the GUI for the user to input their message.  
master = Tk()
Label(master, text="Enter your message:").grid(row=0)

mesg = Entry(master)

mesg.grid(row=0, column=1)

Button(master, text='Quit', command=master.destroy).grid(row=3, column=0, sticky=W, pady=4)
Button(master, text='Enter', command=Recomend_Emoji).grid(row=3, column=1, sticky=W, pady=4)

mainloop( )

Recomended emoji:  😂
